### Cycling & Degradation Profile

This file allows the user to generate a visualisation of cycling and degradation of the BESS.

1. Inputs
raw_output_filename:str - the model output filename that the user wants to analyse
random_day:pd.datetime - the date the user desires to view

2. Outputs
figure:plotlyfig - a plotly figure

In [113]:
import os 
import pandas as pd
import plotly.express as px
import nbformat
import plotly.graph_objects as go

# set some default colors
zenobe_grey = "#9B9A9A"
zenobe_red = "#F37676"
zenobe_black = "#000000"
zenobe_blue = "#4d76ff"
zenobe_green ="#008000"

graph_primary = "rgba(230,98,5,1)" 
graph_primary_med = "rgba(230,98,5,0.75)" 
graph_primary_light = "rgba(230,98,5,0.4)"
graph_primary_v_light = "rgba(230,98,5,0.1)"
graph_secondary = "rgb(144,144,144)"
graph_secondary_med = "rgba(144,144,144,0.75)" 
graph_secondary_light = "rgba(144,144,144,0.5)"
graph_secondary_v_light = "rgba(144,144,144,0.2)"


font_size = 14
font_family = "Century Gothic"
num_time_periods = 48
font_color = "black"
project_name = "Example"
logo_filename = "zenobe_logo.png"
project_power_capacity = 100
project_energy_capacity = 100



In [114]:
# Read in data
# Get the current directory of the script
current_directory = os.path.dirname(os.path.abspath(os.getcwd()))

# Get the parent directory
parent_directory = os.path.dirname(current_directory)
raw_output_filename = "example_02-07.csv"
optimiser_output_df = pd.read_csv(os.path.join(parent_directory,"optimisation","data_output","raw_output",raw_output_filename),index_col=[0])
optimiser_output_df.index = pd.to_datetime(optimiser_output_df.index)
optimiser_output_df

,settlement_period,day_ahead_price,degraded_energy_capacity,intraday_price,import_da_vol,import_bess,export_da_vol,export_bess,soc,da_export_throughput,...,energy_flow_export_da_vol,energy_flow_export_intraday_bess_vol,energy_flow_import_da_vol,energy_flow_import_intraday_bess_vol,cash_flow_export_da_vol,cash_flow_export_intraday_bess_vol,cash_flow_import_da_vol,cash_flow_import_intraday_bess_vol,cash_flow_bess_da,cash_flow_bess_intraday
2023-01-15 00:00:00,1,129.94,100.00,4.12,0.0,1.0,0.000,0.0,0.000,0.000,...,0.000,0.000000e+00,0.0,0.000000,0.00000,0.000000e+00,-0.0,-0.000000,0.00000,0.000000e+00
2023-01-15 00:30:00,2,129.94,100.00,12.81,0.0,1.0,0.000,0.0,0.000,0.000,...,0.000,0.000000e+00,0.0,23.529412,0.00000,0.000000e+00,-0.0,-354.602076,0.00000,-3.546021e+02
2023-01-15 01:00:00,3,47.06,100.00,18.95,0.0,1.0,0.000,0.0,0.000,0.000,...,0.000,0.000000e+00,0.0,0.000000,0.00000,0.000000e+00,-0.0,-0.000000,0.00000,0.000000e+00
2023-01-15 01:30:00,4,47.06,100.00,65.55,0.0,1.0,0.000,0.0,0.000,0.000,...,0.000,0.000000e+00,0.0,0.000000,0.00000,0.000000e+00,-0.0,-0.000000,0.00000,0.000000e+00
2023-01-15 02:00:00,5,39.40,100.00,102.43,0.0,1.0,0.000,0.0,0.000,0.000,...,0.000,0.000000e+00,0.0,0.000000,0.00000,0.000000e+00,-0.0,-0.000000,0.00000,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-30 21:30:00,44,161.94,99.76,64.22,0.0,0.0,39.904,1.0,79.808,19.952,...,19.952,3.552714e-15,0.0,23.472941,3231.02688,2.281553e-13,-0.0,-1773.449744,3231.02688,-1.773450e+03
2023-01-30 22:00:00,45,190.26,99.76,76.03,0.0,0.0,39.904,1.0,59.856,19.952,...,19.952,3.552714e-15,0.0,0.000000,3796.06752,2.701128e-13,-0.0,-0.000000,3796.06752,2.701128e-13
2023-01-30 22:30:00,46,190.26,99.76,116.75,0.0,0.0,39.904,1.0,39.904,19.952,...,19.952,3.552714e-15,0.0,0.000000,3796.06752,4.147793e-13,-0.0,-0.000000,3796.06752,4.147793e-13
2023-01-30 23:00:00,47,190.00,99.76,139.73,0.0,0.0,39.904,1.0,19.952,19.952,...,19.952,3.552714e-15,0.0,0.000000,3790.88000,4.964207e-13,-0.0,-0.000000,3790.88000,4.964207e-13


In [115]:
### Check how many cycles the BESS is doing daily
num_days = optimiser_output_df.index.day.nunique()
bess_cycle_df = optimiser_output_df[["intraday_export_throughput","degraded_energy_capacity"]].copy()
bess_cycle_df["num_cycles"] = bess_cycle_df["intraday_export_throughput"]/project_energy_capacity
bess_cycle_grouped_df = bess_cycle_df.groupby([(bess_cycle_df.index.date)]).agg(total_num_cycles = ("num_cycles","sum"), degraded_energy_capacity = ("degraded_energy_capacity","mean"))
bess_cycle_df

,intraday_export_throughput,degraded_energy_capacity,num_cycles
2023-01-15 00:00:00,0.000,100.00,0.00000
2023-01-15 00:30:00,0.000,100.00,0.00000
2023-01-15 01:00:00,0.000,100.00,0.00000
2023-01-15 01:30:00,0.000,100.00,0.00000
2023-01-15 02:00:00,0.000,100.00,0.00000
...,...,...,...
2023-01-30 21:30:00,19.952,99.76,0.19952
2023-01-30 22:00:00,19.952,99.76,0.19952
2023-01-30 22:30:00,19.952,99.76,0.19952
2023-01-30 23:00:00,19.952,99.76,0.19952


In [116]:

from plotly.subplots import make_subplots
fig = make_subplots(rows=1,cols=1,specs=[[{"secondary_y": True}]])


In [117]:

from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])
trace1 = go.Bar(x=bess_cycle_grouped_df.index,
                y=bess_cycle_grouped_df["total_num_cycles"],
                name='Number of Cycles Per Day',
                marker_color=graph_primary,
                yaxis='y2'
                    )
trace2 = go.Scatter(x=bess_cycle_grouped_df.index,
                        y=bess_cycle_grouped_df["degraded_energy_capacity"],
                        name='BESS Energy Capacity',
                        mode='lines',
                        marker_color=graph_secondary,
                        line=dict(width=2, 
                        color=graph_secondary, 
                        dash='dash'),
                        yaxis='y1')



data = [trace1,trace2]

fig.update_layout(title=dict(x = 0.5,
                            y = 0.95,
                            xanchor =  'center',
                            yanchor = 'top',text=f"{project_name} Mean Number of Cycles & Degraded Energy Capacity<br><sup>{project_power_capacity}MW/{project_energy_capacity}MWh BESS"),
                    yaxis=dict(title='Degraded Energy Capacity (MWh)'),
                    yaxis2=dict(title='Mean Number of Cycles Per Day',
                                
                                side='right'),
)



fig.update_layout( 
    xaxis_title="Year",
)


fig.update_yaxes(range=[0,3], secondary_y=True)   
fig.update_layout(
    yaxis_range=[0,project_energy_capacity],
    barmode="overlay",
    legend=dict(
    orientation="h",
    yanchor="top",
    y=1.25,
    xanchor="right",
    x=1,),
    paper_bgcolor='rgb(256,256,256)',
    plot_bgcolor='rgb(256,256,256)',
    font=dict(
        family=font_family,
        size=font_size,
        color= "black",
    ),
)
fig.update_layout( 
    
    xaxis=dict(
        title='Year',
        tickangle=-45  # Rotate xticks by -45 degrees
    ),
)
for trace in data:
    fig.add_trace(trace) 
fig.update_xaxes(type='category')

fig.update_layout(
    margin=dict(b=140),
    annotations=[
    
        go.layout.Annotation(
            x=0,  # X position on the axis (0 is the far left, 1 is far right)
            y=-0.38,  # Y position below the plot (negative value to place it below)
            xref='paper',  # 'paper' makes the annotation relative to the entire plot (not data coordinates)
            yref='paper',
            text=f"Notes: Intraday churning is not assumed.",  # The note text
            showarrow=False,  # No arrow
            xanchor='left',  # Align the text to the left
            yanchor='top',  # Anchor the note to the top of the y-position
            font=dict(size=font_size)  # Customize the font size if needed
        ),
        
    ]
)
fig.add_layout_image(
    dict(
        source="images/zenobe_logo.PNG",  # Path to the logo
        xref="paper", yref="paper",
        x=1.08, y=1.3,  # Positioning the logo at the top right
        sizex=.2, sizey=.2,  # Adjust size based on your needs
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

In [118]:


trace1 = go.Bar(x=random_period.index,
                y=-random_period["import_intraday_vol"],
                name="Intraday",
                marker_color=graph_secondary,
                yaxis='y1'
                    )


trace2 = go.Bar(x=random_period.index,
                y=random_period["export_intraday_vol"],
                name='Export Intraday',
                marker_color=graph_secondary,
                yaxis='y1',
                showlegend=False
                    )
trace3 = go.Scatter(x=random_period.index,
                        y=random_period["intraday_price"],
                        name='Intraday Price',
                        mode='lines',
                        marker_color=graph_secondary,
                        line=dict(width=2, 
                        color=graph_secondary, 
                        dash='dash'),
                        yaxis='y2')
data = [trace1,trace2]

fig.update_layout(barmode='stack')

fig.update_yaxes(title_text="Power (MW)",row=2,col=1)   
fig.update_yaxes(title_text= "Price (£/MWh)",row=2,col=1,secondary_y=True) 
for trace in data:
    fig.add_trace(trace,row=2,col=1)
fig.add_trace(trace3,row=2,col=1,secondary_y=True)


Exception: The (row, col) pair sent is out of range. Use Figure.print_grid to view the subplot grid. 

In [8]:
#SoC
trace1 = go.Scatter(x=random_period.index,
                        y=(random_period["soc_intraday"]/project_energy_capacity)*100,
                        name='SoC (%)',
                        mode='lines',
                        marker_color=graph_secondary,
                        line=dict(width=2, 
                        color='black', 
                        dash='dash'),
                        yaxis='y2')

soc_data = [trace1]


fig.update_yaxes(title_text= "SoC (%)",range=[0,110], row=3,col=1,secondary_y=False) 


for trace in soc_data:
    fig.add_trace(trace,row=3,col=1)

In [9]:
fig.update_layout(barmode='overlay',
                  font=dict(
        family=font_family,
        size=font_size,
        color= zenobe_black,
    ),)
# Update the layout to show time on x-axis

fig.update_layout(title=dict(x = 0.5,
                            xanchor =  'center',
                            yanchor = 'top',text=f"Example Operation from {random_period_start.date()} to {random_period_end.date()}<br><sup>{project_power_capacity}MW/{project_energy_capacity}MWh BESS, 2 cycles max"),
                    
)
fig.update_layout(
    margin=dict(b=10),
    annotations=[
        go.layout.Annotation(
            x=0,  # X position on the axis (0 is the far left, 1 is far right)
            y=-0.2,  # Y position below the plot (negative value to place it below)
            xref='paper',  # 'paper' makes the annotation relative to the entire plot (not data coordinates)
            yref='paper',
            text=f"Notes: Assumes perfect forecast at Day Ahead, with cycling reserved for intraday cycling",  # The note text
            showarrow=False,  # No arrow
            xanchor='left',  # Align the text to the left
            yanchor='top',  # Anchor the note to the top of the y-position
            font=dict(size=14)  # Customize the font size if needed
        ),
        
    ]
)
fig.add_layout_image(
    dict(
        source="images/zenobe_logo.PNG",  # Path to the logo
        xref="paper", yref="paper",
        x=1.2, y=1.05,  # Positioning the logo at the top right
        sizex=.25, sizey=.25,  # Adjust size based on your needs
        xanchor="right", yanchor="bottom"
    )
)
fig.update_layout(
    paper_bgcolor='rgb(256,256,256)',
    plot_bgcolor='rgb(256,256,256)',
    font_color="black",
    barmode="relative",
    width=1000,
    height=600,
)
fig.show()